<a href="https://colab.research.google.com/github/maciejskorski/ml_examples/blob/master/AttritionModel_TF_UserAPI/ModelServing_Interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook demonstrates how to serve a machine-learning model (trained with Tensorflow software) and make real-time predictions with user-specified input.
The model aims to predict the employee attrition given some personal information.

# Prerequisites

We download the pretrained model and Tensorflow Serving software.

In [1]:
#@title Download the model

!apt-get -q install subversion
!svn checkout https://github.com/maciejskorski/ml_examples/trunk/AttritionModel_TF_UserAPI/attrition_model/
!mv attrition_model /tmp

Reading package lists...
Building dependency tree...
Reading state information...
subversion is already the newest version (1.9.7-4ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
svn: E155004: Run 'svn cleanup' to remove locks (type 'svn help cleanup' for details)
svn: E155004: Working copy '/content/attrition_model' locked.
svn: E155004: '/content/attrition_model' is already locked.
mv: cannot move 'attrition_model' to '/tmp/attrition_model': Directory not empty


In [2]:
#@title Install Tensorflow Serving 
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl -s https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt -qq update
!apt-get install -q tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
OK
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
tensorflow-model-server is already the newest version (2.5.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


# Serving Model

Run the model and check it status 

In [3]:
#@title Start Job
%%bash --bg 
nohup tensorflow_model_server --rest_api_port=8501 --model_name=attrition_model --model_base_path="/tmp/attrition_model" >server.log 2>&1

#!ps aux | grep -i "tensorflow_model_server"
#!kill -9 1971

Starting job # 0 in a separate thread.


In [4]:
#@title Logs and Status

!tail server.log
!curl http://localhost:8501/v1/models/attrition_model

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


# Real-Time Predictions

The model will do predictions (attrition chances from 0 to 100%) based on user input (employee personal information). 

Play with the form below! How overtime and gender affects the chance of attrition?


In [6]:
#@title User Input (Friendly API)

import json, requests

## grab user data (interactive!)

Age = 41 #@param 
BusinessTravel = 'Travel_Frequently' #@param ['Travel_Rarely', 'Travel_Frequently', 'Non-Travel']
DailyRate =  1102 #@param 
Department = 'Sales' #@param ['Sales', 'Research & Development', 'Human Resources']
DistanceFromHome = 1 #@param
Education =  2#@param 
EducationField = 'Human Resources' #@param ['Life Sciences', 'Other', 'Medical', 'Marketing','Technical Degree', 'Human Resources']
EnvironmentSatisfaction = 2 #@param
Gender = 'Male' #@param ['Female', 'Male']
JobInvolvement = 3 #@param
JobLevel = 2 #@param
JobRole = 'Sales Executive' #@param ['Sales Executive', 'Research Scientist', 'Laboratory Technician','Manufacturing Director','Healthcare Representative','Manager','Sales Representative', 'Research Director', 'Human Resources']
JobSatisfaction = 4 #@param
MaritalStatus = 'Single' #@param ['Single', 'Married', 'Divorced']
MonthlyIncome = 5993 #@param
MonthlyRate = 19479 #@param
NumCompaniesWorked = 8 #@param
OverTime = 'No' #@param ['Yes','No']
PercentSalaryHike = 11 #@param 
RelationshipSatisfaction = 1 #@param
StockOptionLevel = 0 #@param
TotalWorkingYears = 8 #@param
TrainingTimesLastYear = 0 #@param
WorkLifeBalance =  2#@param
YearsAtCompany =  1#@param
YearsInCurrentRole =  1#@param
YearsSinceLastPromotion = 0 #@param
YearsWithCurrManager =  1#@param

data = dict(
  Age=Age,
  BusinessTravel=BusinessTravel,
  DailyRate=DailyRate,
  Department=Department,
  DistanceFromHome=DistanceFromHome,
  Education=Education,
  EducationField=EducationField,
  EnvironmentSatisfaction=EnvironmentSatisfaction,
  Gender=Gender,
  JobInvolvement=JobInvolvement,
  JobLevel=JobLevel,
  JobRole=JobRole,
  JobSatisfaction=JobSatisfaction,
  MaritalStatus=MaritalStatus,
  MonthlyIncome=MonthlyIncome,
  MonthlyRate=MonthlyRate,
  NumCompaniesWorked=NumCompaniesWorked,
  OverTime=OverTime,
  PercentSalaryHike=PercentSalaryHike,
  RelationshipSatisfaction=RelationshipSatisfaction,
  StockOptionLevel=StockOptionLevel,
  TotalWorkingYears=TotalWorkingYears,
  TrainingTimesLastYear=TrainingTimesLastYear,
  WorkLifeBalance=WorkLifeBalance,
  YearsAtCompany=YearsAtCompany,
  YearsInCurrentRole=YearsInCurrentRole,
  YearsSinceLastPromotion=YearsSinceLastPromotion,
  YearsWithCurrManager=YearsWithCurrManager
)

## serialize data

js = [{k:[v] for k,v in data.items()}]
js = json.dumps({"signature_name": "serving_default", "instances": js})

## prepare and send the request

headers = {"content-type": "application/json"}
js_response = requests.post('http://localhost:8501/v1/models/attrition_model:predict', data=js, headers=headers)

## look at the response
json.loads(js_response.text)

{'predictions': [[0.70955646]]}